In [1]:
# import libraries

import subprocess
import json
import os
from constants import *
from dotenv import load_dotenv
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy



In [3]:
# Loading environment variable

load_dotenv()

True

In [4]:
# get the saved mnemonic

mnemonic = os.getenv('mnemonic')
print(mnemonic)

duck wolf scissors inside gain squeeze scan tenant clip run improve pool


In [5]:
# connect to local ETH/ geth

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)


In [6]:
# Define function to derive wallet addresses.
def derive_wallets(coin_name):
    
    command = 'php hd-wallet-derive/derive -g --mnemonic="duck wolf scissors inside gain squeeze scan tenant clip run improve pool" --numderive=3 --coin="{}" --cols=index,path,address,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'.format(coin_name)
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()

    return json.loads(output)
    

In [17]:
# create list of coins to be derived using declarations from constants.py

coin_name = [ETH,BTCTEST]
coins={}
for coin in coin_name:
    coins[coin] = derive_wallets(coin)
coins

{'eth': [{'index': 0,
   'path': "m/44'/60'/0'/0/0",
   'address': '0x9f8A6d6Eb0887ddDE9B3F8b3c12f40BBCCeCDFf0',
   'privkey': '0xb42b64e59cebc07279d35297047ebdace9d8ec0a6f0052fe0e3d6f9d695b5319',
   'pubkey': '031165a3d5d69634a32f2b223cad2fc15f1e1cb166573dc9a3f8a0cd28c3135948',
   'pubkeyhash': '2efb6a3388630811684aee873f6d28b761c88e4c',
   'xprv': 'xprvA2pGR2oCcpuRatjsKwbJgQ2yxjk84sugE73YgUADHdWT79bPT8XgohubvUFHHUXhgdcWzTGYg7kytXHDsvjNUP9R5DcT9DtBP5U34y8QqgF',
   'xpub': 'xpub6FocpYL6TCTioNpLRy8K3XyiWmacULdXbKy9UrZpqy3RywvXzfqwMWE5mjz76gogHRC1Qc5cxpTRfqKEyr1URUM581soW4dqSxuXCbqZJTT'},
  {'index': 1,
   'path': "m/44'/60'/0'/0/1",
   'address': '0x2a98A6dEce32591A5d1fF93be3F0953bC876A733',
   'privkey': '0xd5b72be4deceb7ea902fec7a230629d9a2969856e7537cf0e22ce347bd9ad956',
   'pubkey': '022a40027c23d029ab6d434a30fa63332d719ef9a7fb67f2c4ab72bb8bfddf2751',
   'pubkeyhash': 'bb98d22f559ddffd0dfb102ce91ddc5a50283936',
   'xprv': 'xprvA2pGR2oCcpuRc286EPv3EjgGJbdRa9bARSBQmpt99TpgnYabPN9HFQZn

In [18]:
#create a function that convert the privkey string in a child key to an account object

def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)


In [19]:
# Function to create transactions

def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(btc_acc.address, [(recipient, amount, BTC)])

In [20]:
#Function to send transactions
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)


In [21]:
# Declaring variables to store private key

eth_acc = priv_key_to_account(ETH, coins[ETH][0]['privkey'])
btc_acc = priv_key_to_account(BTCTEST,coins[BTCTEST][0]['privkey'])

eth
0xb42b64e59cebc07279d35297047ebdace9d8ec0a6f0052fe0e3d6f9d695b5319
btc-test
cT37rRHY2vsfF7DrkWf8Gn2xD2MjN1FBpGTudgghjBgneMVk6TaM


In [23]:
#Creating transaction in ETH

create_tx(ETH,eth_acc,"0xf096a93fAaCD15848F3CF94ACc0bd3f7d8F5c808", 1)

{'from': '0x9f8A6d6Eb0887ddDE9B3F8b3c12f40BBCCeCDFf0',
 'to': '0xf096a93fAaCD15848F3CF94ACc0bd3f7d8F5c808',
 'value': 1,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 15}

In [24]:
#Sending the transaction to designated network

send_txn(ETH, eth_acc,"0x3dA0F09EF6F833b179768a63AdD29061e0702e47", 1)

0xf2044afd27a7de9c125268d85267f7bbbd0f31e4db61136d19f2e5236b2accee


'0xf2044afd27a7de9c125268d85267f7bbbd0f31e4db61136d19f2e5236b2accee'

In [25]:
#checking balance of the account

w3.eth.getBalance("0x3dA0F09EF6F833b179768a63AdD29061e0702e47")

11

In [27]:
#Creating transaction in BTC

create_tx(BTCTEST,btc_acc,"mvLxU6Do3d5cHRm5r9yGhg9c1DtRvRww5j", 0.00000001)

'{"unspents":[{"amount":1000,"confirmations":3,"script":"76a914a2a5df20abf0f3ce2ebf1673b84eef3a7ca20e5288ac","txid":"5213aa536a827da94ad9c2f81243c2c485186396681960b21f8bd29534448ef6","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":37800,"confirmations":3,"script":"76a914a2a5df20abf0f3ce2ebf1673b84eef3a7ca20e5288ac","txid":"5213aa536a827da94ad9c2f81243c2c485186396681960b21f8bd29534448ef6","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":100000,"confirmations":0,"script":"76a914a2a5df20abf0f3ce2ebf1673b84eef3a7ca20e5288ac","txid":"d55fcb3dd29ed24d9c79e0dc80e3a6fbcd7a040b3308f52ed56bc67baa5ede8d","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mvLxU6Do3d5cHRm5r9yGhg9c1DtRvRww5j",1],["mvLxU6Do3d5cHRm5r9yGhg9c1DtRvRww5j",85555]]}'

In [28]:
send_txn(BTCTEST,btc_acc,"mvLxU6Do3d5cHRm5r9yGhg9c1DtRvRww5j", 0.00000001)

0100000003f68e443495d28b1fb260196896631885c4c24312f8c2d94aa97d826a53aa1352000000006a47304402200dd8cc6764df0b97e750f0a5caaaa6ac02dcb578a8c06eb8ed1bba270ef02fb1022065780b31613422707b97a30599fa539f22d4ddac01916b74e23a89cd96350e47012102d2a406c8908541038ee3df8f7fd6ec53d4749d378358215cc98200d15e9476e0fffffffff68e443495d28b1fb260196896631885c4c24312f8c2d94aa97d826a53aa1352010000006b483045022100872818b39f40006371f45eec7723a78b51f204bf16dfd249ba81dfd89d0c86a50220217cd588052c1a06ba67cc2a3380c105f6146b87247c03720566622d541f664f012102d2a406c8908541038ee3df8f7fd6ec53d4749d378358215cc98200d15e9476e0ffffffff8dde5eaa7bc66bd52ef508330b047acdfba6e380dce0799c4dd29ed23dcb5fd5000000006b4830450221009c383179d537c4eaf46d79996cb09e24846860743fc1f28ab0be26851c2b374a02206f8c6e29564b5af9891f5f4b6b9f154f89363b7c1f07cf1337c675aaef00ff70012102d2a406c8908541038ee3df8f7fd6ec53d4749d378358215cc98200d15e9476e0ffffffff0201000000000000001976a914a2a5df20abf0f3ce2ebf1673b84eef3a7ca20e5288ac334e0100000000001976a914a2a5df20ab